In [3]:
import pandas as pd

In [4]:
sales = pd.read_csv("../DataSet/start_vgsales.csv")
steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]
steam.columns = ["player_id", "game_title", "type", "value"]

C:\Users\david\AppData\Local\Temp\ipykernel_5276\560633259.py:2: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]


In [5]:
steam = steam[steam["type"] == "purchase"]

In [6]:
df = steam.merge(sales, how='inner',left_on="game_title", right_on="Name")
df.head()

,player_id,game_title,type,value,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,151603712,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
1,151603712,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09
2,151603712,Fallout 4,purchase,1.0,1632,Fallout 4,PC,2015.0,Role-Playing,Bethesda Softworks,0.50,0.63,0.00,0.10,1.23
3,87445402,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
4,87445402,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09


In [7]:
df2 = df[["player_id", "Genre", "value"]]
df3 = df2.groupby(["player_id", "Genre"]).count()

In [176]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler

In [177]:
scaler = StandardScaler()
users = df3.reset_index()
users["player_id"] = np.asarray(users["player_id"]).astype(np.int32)
users[["value"]] = scaler.fit_transform(users[["value"]])
unique_user_ids = users["player_id"].unique()
users

,player_id,Genre,value
0,86540,Action,0.173283
1,86540,Adventure,-0.432836
2,86540,Racing,-0.069164
3,86540,Shooter,2.355311
4,86540,Strategy,-0.190388
...,...,...,...
8953,99723205,Shooter,0.779402
8954,99723205,Strategy,-0.554059
8955,99766416,Shooter,-0.432836
8956,99812428,Strategy,-0.432836


In [178]:
genres = users["Genre"].unique()
unique_genres = genres
genres

array(['Action', 'Adventure', 'Racing', 'Shooter', 'Strategy',
       'Role-Playing', 'Simulation', 'Sports', 'Fighting', 'Misc',
       'Puzzle', 'Platform'], dtype=object)

In [179]:
tf.random.set_seed(42)
users_tensor = tf.data.Dataset.from_tensor_slices(dict(users))
users_shuffled = users_tensor.shuffle(seed=42, buffer_size=100_000, reshuffle_each_iteration=False)

users_train = users_shuffled.take(8_000)
users_test = users_shuffled.skip(8_000).take(958)
print(len(users_test), users_test)

958 <TakeDataset element_spec={'player_id': TensorSpec(shape=(), dtype=tf.int32, name=None), 'Genre': TensorSpec(shape=(), dtype=tf.string, name=None), 'value': TensorSpec(shape=(), dtype=tf.float64, name=None)}>


In [180]:
genres_tensor = tf.data.Dataset.from_tensor_slices(genres)

In [181]:
for r in genres_tensor.take(5):
    print(r)

tf.Tensor(b'Action', shape=(), dtype=string)
tf.Tensor(b'Adventure', shape=(), dtype=string)
tf.Tensor(b'Racing', shape=(), dtype=string)
tf.Tensor(b'Shooter', shape=(), dtype=string)
tf.Tensor(b'Strategy', shape=(), dtype=string)


In [182]:
from typing import Dict, Text

In [183]:
class RankingModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.
  def __init__(
          self,
          user_model: tf.keras.Model,
          genre_model: tf.keras.Model,
  ):
    super().__init__()

      # Set up user and movie representations.
    self.user_model = user_model
    self.genre_model = genre_model

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
    ])

  def call(self, inputs):
    player_id, genre = inputs

    user_embedding = self.user_model(player_id)
    genre_embedding = self.genre_model(genre)

    return self.ratings(tf.concat([user_embedding, genre_embedding], axis=1))


In [184]:
class MovielensModel(tfrs.models.Model):

  def __init__(
    self,
    user_model: tf.keras.Model,
    genre_model: tf.keras.Model
  ):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel(user_model, genre_model)
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["player_id"], features["Genre"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("value")
    
    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)


In [195]:
user_model = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(mask_token=None, vocabulary=unique_user_ids),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32)
])
genre_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(mask_token=None, vocabulary=unique_genres),
    tf.keras.layers.Embedding(len(unique_genres) + 1, 32)
])

In [196]:
RankingModel(user_model, genre_model)(([86540], ["Action"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00555841]], dtype=float32)>

In [197]:
cached_train = users_train.shuffle(100_000).batch(8192).cache()
cached_test = users_test.batch(4096).cache()

In [198]:
model = MovielensModel(user_model, genre_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

# Train for 3 epochs.
model.fit(cached_train, epochs=300)

Epoch 1/300
1/1 [==============================] - 2s 2s/step - root_mean_squared_error: 1.0097 - loss: 1.0195 - regularization_loss: 0.0000e+00 - total_loss: 1.0195
Epoch 2/300
1/1 [==============================] - 0s 47ms/step - root_mean_squared_error: 1.0048 - loss: 1.0097 - regularization_loss: 0.0000e+00 - total_loss: 1.0097
Epoch 3/300
1/1 [==============================] - 0s 74ms/step - root_mean_squared_error: 1.0003 - loss: 1.0005 - regularization_loss: 0.0000e+00 - total_loss: 1.0005
Epoch 4/300
1/1 [==============================] - 0s 34ms/step - root_mean_squared_error: 0.9955 - loss: 0.9910 - regularization_loss: 0.0000e+00 - total_loss: 0.9910
Epoch 5/300
1/1 [==============================] - 0s 46ms/step - root_mean_squared_error: 0.9900 - loss: 0.9800 - regularization_loss: 0.0000e+00 - total_loss: 0.9800
Epoch 6/300
1/1 [==============================] - 0s 72ms/step - root_mean_squared_error: 0.9836 - loss: 0.9675 - regularization_loss: 0.0000e+00 - total_loss: 0

In [199]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 210ms/step - root_mean_squared_error: 0.7021 - loss: 0.4929 - regularization_loss: 0.0000e+00 - total_loss: 0.4929


{'root_mean_squared_error': 0.7020923495292664,
 'loss': 0.4929336607456207,
 'regularization_loss': 0,
 'total_loss': 0.4929336607456207}

In [201]:
test_ratings = {}
test_movie_titles = unique_genres
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "player_id": np.array([97634356]),
      "Genre": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")


Ratings:
Action: [[-0.11224034]]
Racing: [[-0.22170338]]
Shooter: [[-0.25671235]]
Misc: [[-0.30114028]]
Platform: [[-0.30174312]]
Sports: [[-0.30253398]]
Adventure: [[-0.39542428]]
Role-Playing: [[-0.39695773]]
Strategy: [[-0.432605]]
Fighting: [[-0.4664885]]
Puzzle: [[-0.47557607]]
Simulation: [[-0.47860846]]
